In [ ]:
# =========================================================
# DERM7PT DATA LOADER FOR TRAINING NOTEBOOKS
# =========================================================
import os
import json
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms

# =========================================================
# PATHS TO PREPROCESSED DATA
# =========================================================
PREPROCESSED_DIR = r"augmented"

TRAIN_CSV = os.path.join(PREPROCESSED_DIR, "train_metadata_final.csv")
VAL_CSV   = os.path.join(PREPROCESSED_DIR, "val_metadata_final.csv")
TEST_CSV  = os.path.join(PREPROCESSED_DIR, "test_metadata_final.csv")

INFO_PATH = os.path.join(PREPROCESSED_DIR, "preprocessing_info.json")

# =========================================================
# LOAD PREPROCESSED DATA
# =========================================================
print("Loading preprocessed Derm7pt data...")

# Load CSVs
train_df = pd.read_csv(TRAIN_CSV)
val_df   = pd.read_csv(VAL_CSV)
test_df  = pd.read_csv(TEST_CSV)

# Load preprocessing info
with open(INFO_PATH, "r") as f:
    preprocessing_info = json.load(f)

categorical_cols = preprocessing_info["categorical_cols"]
label_mapping = preprocessing_info["label_mapping"]

# Extract class names from label_mapping (sorted by label index)
class_names = [k for k, v in sorted(label_mapping.items(), key=lambda x: x[1])]

print(f"\n✅ Training samples:   {len(train_df)}")
print(f"✅ Validation samples: {len(val_df)}")
print(f"✅ Test samples:       {len(test_df)}")
print(f"\nLabel mapping: {label_mapping}")
print(f"Class names: {class_names}")

# =========================================================
# EXTRACT FEATURES AND LABELS
# =========================================================
def extract_features(df):
    """Extract image paths, metadata features, and labels from dataframe"""
    img_paths = df["ImagePath"].values
    labels = df["label"].values
    
    # Metadata features (all columns except ImagePath and label)
    metadata_cols = [col for col in df.columns if col not in ["ImagePath", "label"]]
    metadata = df[metadata_cols].values
    
    return img_paths, metadata, labels

X_train_img, X_train_meta, y_train = extract_features(train_df)
X_val_img, X_val_meta, y_val       = extract_features(val_df)
X_test_img, X_test_meta, y_test    = extract_features(test_df)

num_classes = len(label_mapping)
print(f"\nNumber of classes: {num_classes}")

# Get input dimensions
non_feature_cols = ["label", "ImagePath"]
X_train_meta_df = train_df.drop(columns=non_feature_cols)
input_dim_meta = X_train_meta_df.shape[1]
print(f"Metadata input dimension: {input_dim_meta}")

# =========================================================
# PYTORCH DATASET CLASS
# =========================================================
class Derm7ptDataset(Dataset):
    """
    Custom Dataset for Derm7pt with images + metadata
    """
    def __init__(self, img_paths, metadata, labels, transform=None):
        self.img_paths = img_paths
        self.metadata = metadata
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        # Load image
        img_path = self.img_paths[idx]
        try:
            image = Image.open(img_path).convert("RGB")
        except Exception as e:
            # Fallback to black image if loading fails
            print(f"Warning: Failed to load {img_path}, using placeholder")
            image = Image.new("RGB", (224, 224), color="black")
        
        if self.transform:
            image = self.transform(image)
        
        # Get metadata and label
        metadata = self.metadata[idx].astype(np.float32)
        label = int(self.labels[idx])
        
        return image, metadata, label

# =========================================================
# DATA TRANSFORMS
# =========================================================
# Training transforms (with augmentation)
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(30),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Validation/Test transforms (no augmentation)
val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# =========================================================
# CREATE DATASETS
# =========================================================
train_dataset = Derm7ptDataset(X_train_img, X_train_meta, y_train, transform=train_transform)
val_dataset   = Derm7ptDataset(X_val_img, X_val_meta, y_val, transform=val_test_transform)
test_dataset  = Derm7ptDataset(X_test_img, X_test_meta, y_test, transform=val_test_transform)

print(f"\n✅ Created PyTorch Datasets")
print(f"   - Train: {len(train_dataset)} samples")
print(f"   - Val:   {len(val_dataset)} samples")
print(f"   - Test:  {len(test_dataset)} samples")

# =========================================================
# CREATE DATALOADERS
# =========================================================
BATCH_SIZE = 32

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0,  # Set to 0 for Windows
    pin_memory=True)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

print(f"\n✅ Created DataLoaders (batch_size={BATCH_SIZE})")
print(f"   - Train batches: {len(train_loader)}")
print(f"   - Val batches:   {len(val_loader)}")
print(f"   - Test batches:  {len(test_loader)}")

# =========================================================
# EXAMPLE: TEST LOADING A BATCH
# =========================================================
print("\n🔍 Testing batch loading...")
for images, metadata, labels in train_loader:
    print(f"   - Image batch shape:    {images.shape}")
    print(f"   - Metadata batch shape: {metadata.shape}")
    print(f"   - Labels batch shape:   {labels.shape}")
    break

print("\n✅ Derm7pt data loading complete! Ready for training.")

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_cluster import knn_graph
import timm
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# =========================================================
# IMPROVED MODEL (A FIXED TO MATCH B'S BEHAVIOR)
# =========================================================
class EarlyFusionWithGCN(nn.Module):
    def __init__(self, input_dim_meta, num_classes, k=8):
        super().__init__()
        self.k = k

        # --- GCN Layers ---
        self.gcn1 = GCNConv(input_dim_meta, 64)
        self.gcn2 = GCNConv(64, 32)
        self.res_proj = nn.Linear(64, 32)

        # --- metadata → pseudo image ---
        self.meta_to_image = nn.Sequential(
            nn.Linear(32, 56 * 56),
            nn.ReLU(),
            nn.BatchNorm1d(56 * 56),
            nn.Dropout(0.3)
        )

        # --- MobileViT backbone ---
        self.mobilevit = timm.create_model(
            "mobilevit_s.cvnets_in1k",
            pretrained=True,
            num_classes=0
        )

        # --- Modify first conv to accept 4 channels ---
        stem_conv = self.mobilevit.stem.conv
        new_conv = nn.Conv2d(
            4, stem_conv.out_channels,
            kernel_size=stem_conv.kernel_size,
            stride=stem_conv.stride,
            padding=stem_conv.padding,
            bias=stem_conv.bias is not None
        )

        with torch.no_grad():
            # copy RGB weights
            new_conv.weight[:, :3] = stem_conv.weight
            # tiny weight for metadata channel
            new_conv.weight[:, 3:] = stem_conv.weight.mean(dim=1, keepdim=True) * 0.1
            # copy bias if exists
            if stem_conv.bias is not None:
                new_conv.bias = stem_conv.bias.clone()

        self.mobilevit.stem.conv = new_conv

        # keep only first 4 stages
        self.mobilevit.stages = nn.Sequential(
            *list(self.mobilevit.stages.children())[:4]
        )
        self.mobilevit.final_conv = nn.Identity()
        self.mobilevit.head = nn.Identity()

        # --- Post Conv ---
        self.post_conv = nn.Sequential(
            nn.Conv2d(128, 160, kernel_size=1, bias=False),
            nn.BatchNorm2d(160),
            nn.ReLU(inplace=True)
        )

        # --- Classifier ---
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(160, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, img, meta, batch_idx):
        B = meta.size(0)

        # CORRECT: dynamic kNN graph WITHOUT self-loops
        edge_index = knn_graph(meta, k=self.k, batch=batch_idx)

        # GCN + residual
        x1 = F.relu(self.gcn1(meta, edge_index))
        x2 = F.relu(self.gcn2(x1, edge_index) + self.res_proj(x1))

        # Metadata → pseudo-image
        meta_img = self.meta_to_image(x2).view(B, 1, 56, 56)
        meta_img = F.interpolate(meta_img, size=(224, 224), mode="bilinear", align_corners=False)

        # Early fusion (4 channels)
        x = torch.cat([img, meta_img], dim=1)

        # CNN forward
        feats = self.mobilevit.stem(x)
        feats = self.mobilevit.stages(feats)
        feats = self.post_conv(feats)
        feats = self.pool(feats).view(B, -1)

        return self.classifier(feats)
    

from torchinfo import summary


batch_size = 16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = EarlyFusionWithGCN(input_dim_meta, num_classes).to(device)

dummy_img = torch.randn(batch_size, 3, 224, 224).to(device)
dummy_meta = torch.randn(batch_size, input_dim_meta).to(device)
dummy_batch_idx = torch.arange(batch_size).to(device)

summary(
    model,
    input_data=[dummy_img, dummy_meta, dummy_batch_idx],
    col_names=["input_size", "output_size", "num_params", "trainable"],
    col_width=20,
    depth=3
)





In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
import random

# -----------------------------
# Seed Utility
# -----------------------------
def set_random_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# -----------------------------
# Early Stopping Class
# -----------------------------
class EarlyStopping:
    def __init__(self, patience=5, verbose=False, delta=0, path='checkpoint.pt'):
        self.patience = patience
        self.verbose = verbose
        self.delta = delta
        self.path = path
        self.counter = 0
        self.best_loss = None
        self.best_accuracy = -np.inf
        self.early_stop = False
        
    def __call__(self, val_loss, val_acc, model):
        # Monitor validation LOSS for early stopping (lower is better)
        if self.best_loss is None:
            self.best_loss = val_loss
            self.counter = 0
        elif val_loss >= self.best_loss - self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience} (val_loss: {val_loss:.4f}, best: {self.best_loss:.4f})')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            if self.verbose:
                print(f'✅ Validation loss improved ({self.best_loss:.6f} --> {val_loss:.6f})')
            self.best_loss = val_loss
            self.counter = 0
        
        # Save checkpoint based on best ACCURACY
        if val_acc > self.best_accuracy:
            self.save_checkpoint(val_acc, model)

    def save_checkpoint(self, val_acc, model):
        if self.verbose:
            print(f'💾 Validation accuracy improved ({self.best_accuracy:.6f} --> {val_acc:.6f}). Saving model...')
        torch.save(model.state_dict(), self.path)
        self.best_accuracy = val_acc

# -----------------------------
# Training Function
# -----------------------------
def train(model, train_loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(train_loader, desc='Training')
    for images, meta, labels in pbar:
        images, meta, labels = images.to(device), meta.to(device), labels.to(device)
        batch_idx = torch.arange(meta.size(0), device=device).long()
        
        optimizer.zero_grad()
        outputs = model(images, meta, batch_idx)
        loss = criterion(outputs, labels)
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        pbar.set_postfix({
            'loss': f'{running_loss/total:.4f}',
            'acc': f'{100.*correct/total:.2f}%'
        })
    
    return running_loss/len(train_loader), correct/total

# -----------------------------
# Validation Function
# -----------------------------
def validate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, meta, labels in val_loader:
            images, meta, labels = images.to(device), meta.to(device), labels.to(device)
            batch_idx = torch.arange(meta.size(0), device=device).long()
            
            outputs = model(images, meta, batch_idx)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    return running_loss/len(val_loader), correct/total

# -----------------------------
# Training Loop with Scheduler and Checkpoint
# -----------------------------
def train_model_with_scheduler_and_checkpoint(
    model, train_loader, val_loader, optimizer, criterion, device, 
    epochs=100, patience=5, scheduler_patience=3, checkpoint_dir='checkpoints'):
    
    # Create checkpoint directory if it doesn't exist
    os.makedirs(checkpoint_dir, exist_ok=True)
    checkpoint_path = os.path.join(checkpoint_dir, 'best_model.pt')
    
    early_stopping = EarlyStopping(
        patience=patience, 
        verbose=True,
        path=checkpoint_path
    )
    scheduler = ReduceLROnPlateau(
        optimizer, 
        mode='min',  # Monitor loss (lower is better)
        patience=scheduler_patience, 
        verbose=True,
        factor=0.1,
        min_lr=1e-6
    )
    
    history = {
        'train_loss': [], 'val_loss': [],
        'train_acc': [], 'val_acc': [],
        'lr': []
    }
    
    best_model_epoch = None
    
    for epoch in range(epochs):
        print(f'\nEpoch {epoch+1}/{epochs}')
        
        # Training phase
        train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
        
        # Validation phase
        val_loss, val_acc = validate(model, val_loader, criterion, device)
        
        # Update history
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        history['lr'].append(optimizer.param_groups[0]['lr'])
        
        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')
        
        # Update scheduler based on validation loss
        scheduler.step(val_loss)
        
        # Early stopping check
        early_stopping(val_loss, val_acc, model)
        if val_acc > early_stopping.best_accuracy:
            best_model_epoch = epoch + 1
            
        if early_stopping.early_stop:
            print("🛑 Early stopping triggered (validation loss not improving)")
            break
    
    # Load best model
    model.load_state_dict(torch.load(checkpoint_path))
    
    return model, history

# -----------------------------
# Evaluation Function
# -----------------------------
def evaluate_test_metrics(model, test_loader, device):
    model.eval()
    all_labels, all_preds = [], []

    with torch.no_grad():
        for images, metas, labels in test_loader:
            images, metas, labels = images.to(device), metas.to(device), labels.to(device)
            batch_idx = torch.arange(metas.size(0), device=device).long()
            outputs = model(images, metas, batch_idx)
            preds = outputs.argmax(dim=1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    all_labels = np.array(all_labels)
    all_preds = np.array(all_preds)

    acc = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='macro')
    
    return acc, precision, recall, f1

# -----------------------------
# Multi-Seed Training
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
seeds = [42, 123, 569]  
results = {
    "accuracy": [],
    "precision": [],
    "recall": [],
    "f1_score": []
}

best_accuracy = 0.0
best_model_state = None

for seed in seeds:
    print(f"\n{'='*60}")
    print(f"Training with random seed: {seed}")
    print(f"{'='*60}")
    set_random_seed(seed)
    
    # Reinitialize model, optimizer, and criterion
    model = EarlyFusionWithGCN(input_dim_meta, num_classes, k=8).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=0.01)
    criterion = nn.CrossEntropyLoss()
    
    # Train the model
    model, history = train_model_with_scheduler_and_checkpoint(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        optimizer=optimizer,
        criterion=criterion,
        device=device,
        epochs=100,
        patience=5,
        scheduler_patience=3,
        checkpoint_dir='checkpoints'
    )
    
    # Evaluate on test set
    acc, precision, recall, f1 = evaluate_test_metrics(model, test_loader, device)
    print(f"\nSeed {seed} Results:")
    print(f"  Accuracy:  {acc:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall:    {recall:.4f}")
    print(f"  F1 Score:  {f1:.4f}")
    
    # Save metrics
    results["accuracy"].append(acc)
    results["precision"].append(precision)
    results["recall"].append(recall)
    results["f1_score"].append(f1)
    
    # Update the best model
    if acc > best_accuracy:
        best_accuracy = acc
        best_model_state = model.state_dict()

# =========================================================
# SUMMARY OF RESULTS
# =========================================================
print(f"\n{'='*60}")
print("FINAL RESULTS ACROSS ALL SEEDS")
print(f"{'='*60}")
for metric, values in results.items():
    avg = np.mean(values)
    std_dev = np.std(values)
    print(f"{metric.replace('_', ' ').title():12s}: Mean={avg:.4f}, StdDev={std_dev:.4f}")

# Save best model
torch.save(best_model_state, 'DermGCNImageMetadataNoKD_best.pth')
print(f"\n✅ Best model saved with accuracy: {best_accuracy:.4f}")

In [ ]:
# Load and evaluate the best model
best_model = EarlyFusionWithGCN(input_dim_meta, num_classes, k=8).to(device)
best_model.load_state_dict(torch.load('DermGCNImageMetadataNoKD_best.pth'))
best_model.eval()

# Test and generate classification report
from sklearn.metrics import classification_report, confusion_matrix
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

def test(model, loader, device, desc="Testing"):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for imgs, metas, labels in tqdm(loader, total=len(loader), desc=desc, unit="batch"):
            imgs, metas, labels = imgs.to(device), metas.to(device), labels.to(device)
            batch_idx = torch.arange(metas.size(0), device=device).long()
            outputs = model(imgs, metas, batch_idx)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return all_labels, all_preds

# Run test
true_labels, pred_labels = test(best_model, test_loader, device, desc="Evaluating on Test")

# Classification Report
print("\n" + "="*60)
print("CLASSIFICATION REPORT")
print("="*60)
report = classification_report(true_labels, pred_labels, target_names=class_names, digits=4)
print(report)

# Confusion Matrix
conf_matrix = confusion_matrix(true_labels, pred_labels, normalize="true")

plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, cmap="Blues", fmt=".2f",
            xticklabels=class_names,
            yticklabels=class_names,
            cbar_kws={'label': 'Normalized Count'})
plt.xlabel("Predicted Label", fontweight="bold", fontsize=12)
plt.ylabel("True Label", fontweight="bold", fontsize=12)
plt.title("Normalized Confusion Matrix - GCN with Image and Metadata (No KD)", fontweight="bold", fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
import torch
import torch.nn as nn
import time
import numpy as np
from ptflops import get_model_complexity_info

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image_size = (3, 224, 224)

def count_parameters(model, trainable_only=False):
    if trainable_only:
        return sum(p.numel() for p in model.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in model.parameters())

class GCNWrapper(nn.Module):
    """Wraps 2-input (img, meta) + batch_idx models for ptflops"""
    def __init__(self, model, meta_dim):
        super().__init__()
        self.model = model
        self.meta_dim = meta_dim

    def forward(self, x):
        B = x.size(0)
        dummy_meta = torch.randn(B, self.meta_dim).to(x.device)
        batch_idx = torch.arange(B, device=x.device)
        return self.model(x, dummy_meta, batch_idx)

def compute_flops(model, meta_dim):
    wrapper = GCNWrapper(model, meta_dim).to(device)
    with torch.no_grad():
        flops, _ = get_model_complexity_info(
            wrapper,
            image_size,
            as_strings=False,
            print_per_layer_stat=False,
            verbose=False
        )
    return float(flops / 1e9)  # GFLOPs

def measure_gpu_latency(model, meta_dim, runs=200, warmup=30):
    if not torch.cuda.is_available():
        return None, None, None

    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    model.eval().to(device)

    dummy_img = torch.randn(1, *image_size, device=device)
    dummy_meta = torch.randn(1, meta_dim, device=device)
    batch_idx = torch.arange(1, device=device)

    # warmup
    for _ in range(warmup):
        _ = model(dummy_img, dummy_meta, batch_idx)
    torch.cuda.synchronize()

    start_evt = torch.cuda.Event(enable_timing=True)
    end_evt = torch.cuda.Event(enable_timing=True)
    times = []

    for _ in range(runs):
        start_evt.record()
        _ = model(dummy_img, dummy_meta, batch_idx)
        end_evt.record()
        torch.cuda.synchronize()
        times.append(start_evt.elapsed_time(end_evt))  # ms

    times = np.array(times)
    mean = float(times.mean())
    std = float(times.std())
    fps = float(1000.0 / mean)
    return mean, std, fps

def measure_cpu_latency(model, meta_dim, runs=100, warmup=20):
    model_cpu = model.cpu()
    model_cpu.eval()

    dummy_img = torch.randn(1, *image_size)
    dummy_meta = torch.randn(1, meta_dim)
    batch_idx = torch.arange(1)

    for _ in range(warmup):
        _ = model_cpu(dummy_img, dummy_meta, batch_idx)

    times = []
    for _ in range(runs):
        start = time.perf_counter()
        _ = model_cpu(dummy_img, dummy_meta, batch_idx)
        end = time.perf_counter()
        times.append((end - start) * 1000.0)

    times = np.array(times)
    mean = float(times.mean())
    std = float(times.std())
    fps = float(1000.0 / mean)
    return mean, std, fps

def benchmark_model(model, name, meta_dim):
    """Comprehensive model benchmarking"""
    print(f"\n{'='*60}")
    print(f"Benchmarking: {name}")
    print(f"{'='*60}")

    params_m = count_parameters(model, trainable_only=True) / 1e6
    print(f"Parameters: {params_m:.3f} M")

    flops_g = compute_flops(model, meta_dim)
    print(f"FLOPs: {flops_g:.3f} G")

    gpu_mean, gpu_std, gpu_fps = measure_gpu_latency(model, meta_dim)
    if gpu_mean is not None:
        print(f"GPU Latency: {gpu_mean:.3f} ± {gpu_std:.3f} ms  |  FPS: {gpu_fps:.1f}")
    else:
        print("GPU Latency: N/A")

    cpu_mean, cpu_std, cpu_fps = measure_cpu_latency(model, meta_dim)
    print(f"CPU Latency: {cpu_mean:.3f} ± {cpu_std:.3f} ms  |  FPS: {cpu_fps:.1f}")

    return {
        "model": name,
        "params_M": params_m,
        "flops_G": flops_g,
        "gpu_latency_mean_ms": gpu_mean,
        "gpu_latency_std_ms": gpu_std,
        "gpu_fps": gpu_fps,
        "cpu_latency_mean_ms": cpu_mean,
        "cpu_latency_std_ms": cpu_std,
        "cpu_fps": cpu_fps,
    }

# Benchmark the best model
best_model = EarlyFusionWithGCN(input_dim_meta, num_classes, k=8)
best_model.load_state_dict(torch.load("DermGCNImageMetadataNoKD_best.pth", map_location="cpu"))
best_model.to(device).eval()

results_benchmark = benchmark_model(best_model, "GCN with Image and Metadata (No KD)", input_dim_meta)
print("\n✅ Complexity analysis complete!")